* Create example posts to use 


In [1]:
#| default_exp testing

%pip install -q langchain_core
%pip install -q langchain_openai
%pip install -q nbdev


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_core.prompts import PromptTemplate

firstPrompt = PromptTemplate.from_template("""Return one or more json objects together in an array to describe the jobs in a job posting. If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create separate json objects for each role. Here are the key values currently being used to categorize jobs:\n{key_values}\n.Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object.\n
Here is the job posting:\n{job_posting}""")

subsequentPrompt = PromptTemplate.from_template("""You will be returning one or more json objects together in an array to describe the jobs in a job posting. If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create a separate json object for each role. Here are examples of the format of the json object you should return:\n{json_examples}\n Here are the key values currently being used to categorize jobs:\n{key_values}\n.Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object.\n Here is the job posting: \n{job_posting}""")



In [3]:
from hn_jobs_chat.scraper import loadJSON

posts = loadJSON('posts.json')

In [4]:
from langchain_openai import ChatOpenAI

post = posts[0]

msg = firstPrompt.format(job_posting = post['comment'])

model = ChatOpenAI()

response = model.invoke(msg)

In [3]:
import  json
# Parse the JSON string to a Python dictionary
initial_parsed_datum = json.loads(response.content)
parsed_data = initial_parsed_datum

parsed_data

NameError: name 'response' is not defined

In [2]:
import  json
from hn_jobs_chat.scraper import saveJSON

result = {}
# Keys extracted from previous job postings
keys = [ 
    'company',
    'location',
    'industry',
    'target_market',
    'company_status',
    'company_description',
    'company_goal',
    'company_stage',
    'more_company_info',
    'employment_type',
    'remote/local details',
    'job_title',
    'job_description'
    'job_requirements/tech stack',
    'job_soft_skills',
    'product_description',
    'tech_stack',
    'application_process',
    'work_schedule',
    'contact_email',
    'contact_url',
    'additional_urls',
    'additional_notes',
    'compensation'
]

itemStr = map(lambda x: json.dumps(x), parsed_data[0])
examplesStr = " ".join(itemStr)

examplesStr

NameError: name 'parsed_data' is not defined

In [ ]:
postItems = posts[:5]

for i in range(len(postItems)):
    if i != 0:
        msg = subsequentPrompt.format(job_posting = postItems[i]['comment'], json_examples = examplesStr, key_values = ", ".join(keys))

        print(msg)
        resp = model.invoke(msg)
        parsed_datum = json.loads(resp.content)
        for  job in  parsed_datum:
            parsed_data.append(job)

        # We don't care much for the shape of the json object the machine generates, as long as it remains uniform. So, we use the first 5 outputs as examples for the rest.
        if i < 5:
            itemStr = map(lambda x: json.dumps(x), parsed_data)
            examplesStr += " ".join(itemStr)

        if i == 4 or i == 0:
            print('\n\n', msg)
            print ('\n\n', examplesStr)
        # We don't care much for the shape of the json object the machine generates, as long as it remains uniform. So, we use the first 5 outputs as examples for the rest.
        if i < 5:
            itemStr = map(lambda x: json.dumps(x), parsed_data)
            examplesStr += " ".join(itemStr)

        if i == 4 or i == 0:
            print('\n\n', msg)
            print ('\n\n', examplesStr)

        print(parsed_data)


saveJSON('examplePosts.json', parsed_data)

We split the iteration over the request so we don't do it unnecessarily

In [52]:
print(parsed_data)

[[{'job_title': 'Senior Software Engineer', 'company': 'Irth Solutions', 'description': 'At Irth Solutions we build software that improve resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Our systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and a variety of other stakeholders throughout North America.', 'technical_requirements': ['C# development experience using .NET Core, WebAPI, and Entity Framework', 'Experience building browser applications using Angular and Typescript', 'Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning', 'Kubernetes, Docker, Linux'], 'apply_link': 'https://apply.workable.com/irth-solutions/j/33AD66BA3A/'}], {'company': 'Tandem Health', 'job_title': 'Software Engineer', 'location': 'Stockholm, Sweden', 'work_schedule

In [53]:

# let's see how frequently each key appears
tally = {}
keys
for key in keys:
#     result[key] = [datum[key] for datum in parsed_data]
    tally[key] = sum([1 for datum in parsed_data if key in datum])

# result, tally

We add all the gathered keys to the table.

We create a SQL query that posts by our new keys

In [57]:
#| hide
from nbdev import nbdev_export
nbdev_export()